In [2]:
# Importing necessary libraries
import pandas as pd               
import numpy as np                
import matplotlib.pyplot as plt   
import os                         
import joblib                     
import hopsworks                  

from SML.feature_pipeline import nvidia_fg, news_sentiment_fg   #Loading in the nvidia_fg & news sentiment fg
#Making the notebook able to fetch from the .env file
from dotenv import load_dotenv
import os

load_dotenv()


Connected. Call `.close()` to terminate connection gracefully.
Copy your Api Key (first register/login): https://c.app.hopsworks.ai/account/api/generated
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.
         date  1. open  2. high    3. low  4. close    5. volume ticker
0  2024-08-02   103.76   108.72  101.3700    107.27  482027464.0   NVDA
1  2024-08-01   117.53   120.16  106.8104    109.21  523462326.0   NVDA
2  2024-07-31   112.90   118.34  110.8800    117.02  473174182.0   NVDA
3  2024-07-30   111.52   111.99  102.5400    103.73  486833274.0   NVDA
4  2024-07-29   113.69   116.28  111.3000    111.59  248152068.0   NVDA
Index(['date', 'open', 'high', 'low', 'close', 'volume', 'ticker'], dtype='object')
2024-08-05 20:10:11,984 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be 

Uploading Dataframe: 0.00% |          | Rows 0/6228 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: nvidia_stock_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/nvidia_stock_5_offline_fg_materialization/executions
2024-08-05 20:10:22,102 WARNING: DeprecationWarning: Providing event_time as a single-element list is deprecated and will be dropped in future versions. Provide the feature_name string instead.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fg/1073488


Uploading Dataframe: 0.00% |          | Rows 0/66 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: news_sentiment_updated_5_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/564374/jobs/named/news_sentiment_updated_5_offline_fg_materialization/executions


True

In [3]:
#Getting connected to hopsworks 
api_key = os.environ.get('hopsworks_api') 
project = hopsworks.login(api_key_value=api_key) 
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/564374
Connected. Call `.close()` to terminate connection gracefully.


In [4]:
#Defining the function to create feature view

def create_stocks_feature_view(fs, version):

    # Loading in the feature groups
    nvidia_fg = fs.get_feature_group('nvidia_stock', version=1)
    news_sentiment_fg = fs.get_feature_group('news_sentiment_updated', version=1)

    # Defining the query
    ds_query = nvidia_fg.select(['date', 'open', 'ticker'])\
        .join(news_sentiment_fg.select(['sentiment']))

    # Creating the feature view
    feature_view = fs.create_feature_view(
        name='nvidia_stocks_fv',
        query=ds_query,
        labels=['open']
    )

    return feature_view, nvidia_fg

In [5]:
#Creating the feature view
try:
    feature_view = fs.get_feature_view("nvidia_stocks_fv", version=1)
    nvidia_fg = fs.get_feature_group('nvidia_stock', version=1)
except:
    feature_view, nvidia_fg = create_stocks_feature_view(fs, 1)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/564374/fs/560197/fv/nvidia_stocks_fv/version/1


In [ ]:
#Defining a function to get fixed data from the feature view
def fix_data_from_feature_view(df,start_date,end_date):
    df = df.sort_values("date")
    df = df.reset_index()
    df = df.drop(columns=["index"])

    # Create a boolean mask for rows that fall within the date range
    mask = (pd.to_datetime(df['date']) >= pd.to_datetime(start_date)) & (pd.to_datetime(df['date']) <= pd.to_datetime(end_date))
    len_df = np.shape(df)
    df = df[mask] # Use the boolean mask to filter the DataFrame
    print('From shape {} to {} after cropping to given date range: {} to {}'.format(len_df,np.shape(df),start_date,end_date))

    # Get rid off all non-business days
    isBusinessDay, is_open = extract_business_day(start_date,end_date)
    is_open = [not i for i in is_open] # Invert the mask to be able to drop all non-buisiness days

    filtered_df = df.drop(df[is_open].index) # Use the mask to filter the rows of the DataFrame
    print('From shape {} to {} after removing non-business days'.format(np.shape(df),np.shape(filtered_df)))
    print(filtered_df)
    
    return filtered_df